In [1]:
import numpy as np
from matplotlib import pyplot as plt
from IPython import display

from scipy.spatial.distance import cdist
import cv2
from scipy.stats import norm
import time

from IPython import display

np.random.seed(0)

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [ ]:
class grid_env():
    
    def __init__(self):
        self.Nx = 40 # size of the grid map
        self.Ny = 40

        
        x = np.arange(self.Nx) # create x coord array
        y = np.arange(self.Ny) # create y coord array

        self.xx,self.yy = np.meshgrid(x,y) # create matrices of x and y coords, as separate matrices

        # create a box with a small opening in the middle
        

        self.xl = np.random.randint(self.N) # goal location
        self.yl = np.random.randint(self.N)

        self.fv = lambda x,y : np.exp(-1*((x-self.xl)**2+(y-self.yl)**2)) # e^(-dist^2). gaussian proximity reward function
        
        self.f = self.fv(self.xx,self.yy) # reward matrix
        
        self.state = np.random.randint(self.N,size=2) # random initial location
        
        self.action_space = np.array([[0,1],[0,-1],[-1,0],[1,0]]) # up down left right 
        
    # apply action and update the state
    def mm(self,X,u,dt=0.1):
        x = X[0] + u[0]
        y = X[1] + u[1]

        return np.array([x,y])
    
    # step one action
    def step(self,idx):
        u = self.action_space[idx,:]
        new_state = self.mm(self.state,u)
        new_state = np.clip(new_state,0,self.N-1)
        self.state = np.copy(new_state)
        
        # if the gaussian proximity lookup table returns higher than 0.1
        if self.f[new_state[1],new_state[0]] > 0.1:
            feature = 2.0 
        else:
            feature = 0.0 
        
        # return new state, gaussian proximity reward, feature (unused)
        return self.state,self.f[new_state[1],new_state[0]],feature
    
    def reset(self):
        self.state = np.random.randint(self.N,size=2)
        if self.f[self.state[1],self.state[0]] > 0.1:
            feature = 2.0
        else:
            feature = 0.0
        return np.copy(self.state),feature